# 패키지 다운로드 & 버전 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvcc -V
!echo
!gcc --version
!nvidia-smi

In [ ]:
!git clone https://github.com/SwinTransformer/Swin-Transformer-Object-Detection.git
!pip install -r /content/Swin-Transformer-Object-Detection/requirements.txt
!pip install openmim
!mim install mmdet
!pip install tqdm

!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd ..

# 데이터 다운로드 & 오류데이터 수정

In [ ]:
!unzip '/content/drive/MyDrive/input/ocr_ancient.zip' -d /content/data
# !unzip '/content/drive/MyDrive/input/train.zip' -d /content/data
# !unzip '/content/drive/MyDrive/input/valid.zip' -d /content/data

In [ ]:
!cp /content/drive/MyDrive/input/train.json /content/data/train.json
!cp /content/drive/MyDrive/input/valid.json /content/data/valid.json

In [ ]:
# 목판본_043_언해류_대학언해_01
# 목판본_043_언해류_대학언해1_01
# 필사본_104_문학류_대명영렬전2_01
# 필사본_104_문학류_대병영렬전2_01
# 필사본_105_문학류_대명영렬전3_01
# 필사본_105_문학류_대병영렬전3_01
import os
from glob import glob
json_list = glob(os.path.join('/content/data/이미지데이터','*')) # derectory
for i in json_list:
    tmp_list = glob(os.path.join(i,'*'))
    for j in tmp_list: # image파일

        if j.split('/')[-1][:-8] != i.split('/')[-1]:
            dir_name = i.split('/')[-1]
            img_number = j.split('/')[-1][-8:]
            os.rename(j, '/content/data/이미지데이터/'+dir_name+'/'+dir_name+img_number)

# 데이터 전처리

In [2]:
import numpy as np
import pandas as pd
import cv2
import os
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import random

from collections import defaultdict, Counter
import json
from glob import glob
import pycocotools

## seg없이 데이터만들기

In [ ]:
def make_dataset(mode='train'):

    os.makedirs(f'/content/data/{mode}', exist_ok=True)
    dir_list = glob('/content/data/라벨링데이터/*')

    count = 0

    image_idx = 0
    annotation_idx = 0

    images = [] # images.append({'id':image_idx, 'file_name': jf_name+'/'+jf_name+file_name[-4:]+'.png','width':d['Image_Width'], 'height':d['Image_Height']})
    annotations = [] # annotations.append({'id':annotations_idx, 'image_id':image_idx, 'category_id':0,\
                    # 'bbox':(x,y,w,h), 'area':w*h, 'ignore':0,'iscrowd':0,'segmentation':[]})
    for idx, i in tqdm(enumerate(dir_list)): # directory
        print(idx, i)
        dir_name = i.split('/')[-1]
        json_list = glob(os.path.join(i, '*')) # json파일
        if mode == 'train': ################################################# train, valid 비율설정
            iter = json_list[:int(len(json_list)*0.5)]
        else:
            iter = json_list[int(len(json_list)*0.8):]
        for j in iter:
            with open(j, 'r', encoding='utf-8-sig') as jfile:
                d = json.load(jfile)
                image_name = d['Image_filename']
                image_num = image_name[-4:]
                if d['Image_Char_no'] > 400:
                    continue
                
                image = plt.imread(i.replace('라벨링데이터','이미지데이터')+'/'+dir_name+image_num+'.png') # h, w, c

    #             image = np.zeros((int(d['Image_Height']), int(d['Image_Width']), 3), dtype=int)
                h,w,c = image.shape

                ### ----------------- Min, Max 계산 ----------------- ###
                min_y, max_y, min_x, max_x = 1e9, 0, 1e9, 0
                for k in d['Text_Coord']:
                    bbox = k['bbox'] # x,y,w,h
                    min_y, max_y, min_x, max_x = min(min_y, bbox[1]), max(max_y,bbox[1]+bbox[3]),\
                                                min(min_x, bbox[0]), max(max_x,bbox[0]+bbox[2])
                # 자를 지점 결과
                mini_y, maxi_y, mini_x, maxi_x = max(0,min_y-32), min(h,max_y+32), max(0,min_x-32), min(w, max_x+32)
                
                ### ----------------- 글 주변만 자른 이미지 ----------------- ###
                result = image[mini_y:maxi_y, mini_x:maxi_x,:]
                h_, w_, c_ = result.shape
                
                ### ----------------- 가운데를 자르기 전 빈 공간이 있는지 체크 ----------------- ###
                width_check = np.array([0]*w_)

                for k in d['Text_Coord']:
                    bbox = k['bbox']
                    annotate = k['annotate']
                    # pp = image[bbox[1]-mini_y:bbox[1]-mini_y+bbox[3],bbox[0]-mini_x:bbox[0]-mini_x+bbox[2],:]
                    width_check[bbox[0]-mini_x:bbox[0]-mini_x+bbox[2]] += 1
                    # rect = patches.Rectangle((bbox[0]-mini_x,bbox[1]-mini_y),bbox[2],bbox[3], linewidth=3, edgecolor='r', facecolor='none')
                    # ax.add_patch(rect)
                plt.show()
                
                ### ----------------- 빈 공간이 있으면 이미지 분할 ----------------- ###
                cutting = None
                start = None # 빈공간의 처음지점
                end = None # 빈공간의 마지막 지점 --> 치우치지 않게 자르기 위함
                flag = False
                if h_/w_ < 1.0:
                    for n in range(10, -25,-1):
                        if width_check[w_//2 + n] == 0 and flag == False:
                            start = w_//2 + n
                            end = w_//2 + n
                            flag = True
                        elif width_check[w_//2 + n] == 0 and flag == True:
                            end = w_//2 + n
                        elif width_check[w_//2 + n] != 0 and flag == True:
                            end = w_//2 + n
                            break
                        else:
                            continue
                    if start is None or end is None: # 가로가 긴데 못자르면 패스
                        continue
                        
                    cutting = (start+end)//2
                    
                    left_coord = []
                    right_coord = []
                    
                    for k in d['Text_Coord']:
                        bbox = k['bbox']
                        bbox_ = [bbox[0]-mini_x,bbox[1]-mini_y,bbox[2],bbox[3]] # 원래 bbox에서 여백제거한만큼 이동
                        if bbox_[0] > cutting:
                            bbox_[0] -= cutting
                            right_coord.append(bbox_)
                        else:
                            left_coord.append(bbox_)
                    
                    image_left = result[:,:cutting,:]
                    image_right = result[:,cutting:,:]
                    
                    h_left, w_left, _ = image_left.shape
                    h_right, w_right, _ = image_right.shape
                    if 2 > h_left / w_left >= 1: # 자른 후 세로가 길어야 저장
                        plt.imsave(f'/content/data/{mode}/{image_idx:05d}.png', image_left)
                        images.append({'id':image_idx, 'file_name': f'{image_idx:05d}.png',\
                                    'width':w_left, 'height':h_left})
                        for ann in left_coord:
                            if 0 > ann[0] or ann[0]+ann[2] > w_left or 0 > ann[1] or ann[1]+ann[3] > h_left:
                                continue
                            annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                                    'bbox':(ann[0],ann[1],ann[2],ann[3]), 'area':ann[2]*ann[3], 'ignore':0,'iscrowd':0,'segmentation':[]})
                            annotation_idx += 1
                        image_idx += 1
                    if 2 > h_right / w_right >= 1:
                        plt.imsave(f'/content/data/{mode}/{image_idx:05d}.png', image_right)
                        images.append({'id':image_idx, 'file_name': f'{image_idx:05d}.png',\
                                    'width':w_right, 'height':h_right})
                        for ann in right_coord:
                            if 0 > ann[0] or ann[0]+ann[2] > w_left or 0 > ann[1] or ann[1]+ann[3] > h_left:
                                continue
                            annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                                    'bbox':(ann[0],ann[1],ann[2],ann[3]), 'area':ann[2]*ann[3], 'ignore':0,'iscrowd':0,'segmentation':[]})
                            annotation_idx += 1

                        image_idx += 1
                        
                    
                else: # 세로가 길면
                    if h_/w_ > 2 or d['Image_Char_no'] > 200:
                        count += 1
                        continue
                    plt.imsave(f'/content/data/{mode}/{image_idx:05d}.png', result)
                    # fig, ax = plt.subplots(figsize=(8,8))
                    # ax.imshow(result)
                    images.append({'id':image_idx, 'file_name': f'{image_idx:05d}.png',\
                                'width':w_, 'height':h_})
                    for ann in d['Text_Coord']:
                        x,y,w,h = ann['bbox'][:4]
                        if 0 > x or x+w > w_ or 0 > y or y+h > h_:
                            continue
                        annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                                'bbox':(x-mini_x,y-mini_y,w,h), 'area':w*h, 'ignore':0,'iscrowd':0,'segmentation':[]})
                        rect = patches.Rectangle((x-mini_x,y-mini_y),w,h, linewidth=3, edgecolor='r', facecolor='none')
                        # ax.add_patch(rect)
                        annotation_idx += 1
                    image_idx += 1
                    # plt.show()
    return images, annotations

## seg 잇이

In [ ]:
!rm -rf /content/data/train
!rm -rf /content/data/valid
!rm -rf /content/data/train.json
!rm -rf /content/data/valid.json

In [7]:
def decide_min_max(coordinates : dict, h, w):
    min_y, max_y, min_x, max_x = 1e9, 0, 1e9, 0

    for i in coordinates:
        bbox = i['bbox']
        min_y, max_y, min_x, max_x = min(min_y, bbox[1]), max(max_y,bbox[1]+bbox[3]),\
                                    min(min_x, bbox[0]), max(max_x,bbox[0]+bbox[2])
    mini_y, maxi_y, mini_x, maxi_x = max(0,min_y-32), min(h,max_y+32), max(0,min_x-32), min(w, max_x+32)
    return mini_y, maxi_y, mini_x, maxi_x


def make_segmentation(x,y,w,h,image):
    """bbox로부터 segmentation 추출"""
    base = np.zeros(image.shape[:2])
    # box = image[y:y+h,x:x+w,:]
    # rgb_distance = np.abs(box[:,:,0] - box[:,:,1]) + np.abs(box[:,:,1] - box[:,:,2]) + np.abs(box[:,:,2] - box[:,:,0])
    # rgb_mask = np.zeros((h,w))
    # for m in np.argwhere(rgb_distance < 0.7):
    #     rgb_mask[m[0], m[1]] = 1
    
    # box_gray = np.mean(box, axis=2)
    # mean_box = np.mean(box.flatten())
    # under_mean = np.argwhere(box_gray <= mean_box*0.9)
    # zero_arg = np.argwhere(box_gray <= np.min(box_gray.flatten())+0.01)
    # tmp_mask = np.zeros((h,w))
    # for m in under_mean:
    #     tmp_mask[m[0],m[1]] = 1
    # for m in zero_arg:
    #     tmp_mask[m[0],m[1]] = 1
    # total_mask = rgb_mask * tmp_mask
    # if np.sum(total_mask.flatten()) < 4:
    #     return None
    base[y:y+h, x:x+w] = total_mask
    base = np.asfortranarray(base).astype(np.uint8)
    seg = pycocotools.mask.encode(base)
    seg['counts'] = seg['counts'].decode('ascii')
    # seg = np.flip(np.argwhere(total_mask==1),axis=1) + [x,y]
    return seg

def append_annotations(image,image_half,coords,annotations,image_idx,annotation_idx,seg_option,full):
    if image.shape[0]*image.shape[1] > 4000000:
        for ann in coords: ############################################### Annotation
            x,y,w,h = ann[:4]
            if full:
                x, y = x-full[0], y-full[1]
            if 0 > x or x+w > image.shape[1] or 0 > y or y+h > image.shape[0]:
                continue
            seg = None
            if seg_option:
                seg = make_segmentation(x//2,y//2,w//2,h//2,image_half)
                if seg is None:
                    continue
                annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                        'bbox':(x//2,y//2,w//2,h//2), 'area':w//2*h//2, 'ignore':0,'iscrowd':0,'segmentation':seg})
            else:
                annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                        'bbox':(x,y,w,h), 'area':w*h, 'ignore':0,'iscrowd':0,'segmentation':[]})
            annotation_idx += 1
    else:
        for ann in coords: 
            x,y,w,h = ann[:4]
            if 0 > x or x+w > image.shape[1] or 0 > y or y+h > image.shape[0]:
                continue
            seg = None
            if seg_option:
                seg = make_segmentation(x,y,w,h,image)
                if seg is None:
                    continue
                annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                        'bbox':(x,y,w,h), 'area':w*h, 'ignore':0,'iscrowd':0,'segmentation':seg})
            else:
                annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                        'bbox':(x,y,w,h), 'area':w*h, 'ignore':0,'iscrowd':0,'segmentation':[]})
            annotation_idx += 1
    return annotations, annotation_idx

def save_iamge(mode, image_idx, image, w, h):
    plt.imsave(f'/content/data/{mode}/{image_idx:05d}.png', image)
    images.append({'id':image_idx, 'file_name': f'{image_idx:05d}.png',\
                'width':w//2, 'height':h//2})
    return images

def decide_cutting_point(width_check, w_):
    cutting = None
    start, end = None, None # 빈공간의 처음지점, 빈공간의 마지막 지점 --> 치우치지 않게 자르기 위함
    flag = False
    for n in range(10, -25,-1):
        if width_check[w_//2 + n] == 0 and flag == False:
            start = w_//2 + n
            end = w_//2 + n
            flag = True
        elif width_check[w_//2 + n] == 0 and flag == True:
            end = w_//2 + n
        elif width_check[w_//2 + n] != 0 and flag == True:
            break
        else:
            continue
    if start is None: # 가로가 긴데 못자르면 패스
        cutting = None
    else:
        cutting = (start+end)//2
    return cutting

In [ ]:
def make_dataset(mode='train', seg_option=False):
    os.makedirs(f'/content/data/{mode}', exist_ok=True)
    dir_list = glob('/content/data/라벨링데이터/*')

    image_idx = 0
    annotation_idx = 0

    images = [] # {'id':image_idx, 'file_name': '.png','width':d['Image_Width'], 'height':d['Image_Height']}
    annotations = [] # {'id':annotations_idx, 'image_id':image_idx, 'category_id':0,'bbox':(x,y,w,h), 'area':w*h, 'ignore':0,'iscrowd':0,'segmentation':[]}
    for idx, i in tqdm(enumerate(dir_list)): # directory
        dir_name = i.split('/')[-1]
        json_list = glob(os.path.join(i, '*')) # json파일
        if mode == 'train': ############################## train, valid 비율설정
            iter = json_list[:int(len(json_list)*0.8)]
        else:
            iter = json_list[int(len(json_list)*0.8):]
        for j in iter:
            with open(j, 'r', encoding='utf-8-sig') as jfile:
                d = json.load(jfile)
                image_name = d['Image_filename']
                image_num = image_name[-4:]
                if d['Image_Char_no'] > 500:
                    continue      
                image = plt.imread(i.replace('라벨링데이터','이미지데이터')+'/'+dir_name+image_num+'.png') # h, w, c
                h,w,c = image.shape

                mini_y, maxi_y, mini_x, maxi_x = decide_min_max(d['Text_Coord'], h,w)
                
                result = image[mini_y:maxi_y, mini_x:maxi_x,:] ### ----------------- 글 주변만 자른 이미지 ----------------- ###
                h_, w_, c_ = result.shape

                ### ----------------- 가운데를 자르기 전 빈 공간이 있는지 체크 ----------------- ###
                width_check = np.array([0]*w_)
                for k in d['Text_Coord']:
                    bbox = k['bbox']
                    width_check[bbox[0]-mini_x:bbox[0]-mini_x+bbox[2]] += 1
                
                if h_/w_ < 1.0:
                    ### ----------------- 빈 공간이 있으면 이미지 분할 ----------------- ###
                    cutting = decide_cutting_point(width_check, w_)
                    if cutting is None:
                        continue
                    
                    left_coord = []
                    right_coord = []
                    
                    for k in d['Text_Coord']:
                        bbox = k['bbox']
                        bbox_ = [bbox[0]-mini_x,bbox[1]-mini_y,bbox[2],bbox[3]] # 원래 bbox에서 여백제거한만큼 이동
                        if bbox_[0] >= cutting:
                            bbox_[0] -= cutting
                            right_coord.append(bbox_)
                        else:
                            left_coord.append(bbox_)
                    
                    image_left = result[:,:cutting,:]  # 가로로 길면 분할
                    image_right = result[:,cutting:,:]
                    
                    h_left, w_left, _ = image_left.shape
                    h_right, w_right, _ = image_right.shape
                    if 2 > h_left / w_left >= 1: # 왼쪽 이미지에 대해서 자른 후 세로가 길어야 저장
                        if h_left*w_left > 4000000: 
                            image_left_ = cv2.resize(image_left, (w_left//2, h_left//2)) # image, w, h
                            images = save_iamge(mode, image_idx, image_left_, w_left//2, h_left//2)
                        else:
                            images = save_iamge(mode, image_idx, image_left, w_left, h_left)
                        annotations, annotation_idx = append_annotations(image_right,image_right_,left_coord,annotations,image_idx,annotation_idx,seg_option,None)
                        image_idx += 1
                    if 2 > h_right / w_right >= 1: # 오른쪽 이미지에 대해서 자른 후 세로가 길어야 저장
                        if h_right*w_right > 4000000:
                            image_right_ = cv2.resize(image_right, (w_right//2, h_right//2)) # w, h
                            images = save_iamge(mode, image_idx, image_right_, w_right//2, h_right//2)
                        else:
                            plt.imsave(f'/content/data/{mode}/{image_idx:05d}.png', image_right)
                            images = save_iamge(mode, image_idx, image_left, w_left, h_left)
                        annotations, annotation_idx = append_annotations(image_right,image_right_,right_coord,annotations,image_idx,annotation_idx,seg_option,None)
                        image_idx += 1
                else: # 세로가 길면
                    if h_/w_ > 2 or d['Image_Char_no'] > 250:
                        continue
                    if h_*w_ > 4000000: ######################################## 이미지
                        result_ = cv2.resize(result, (w_//2, h_//2))
                        images = save_iamge(mode, image_idx, result_, w_//2, h_//2)
                    else:
                        images = save_iamge(mode, image_idx, result, w_left, h_left)
                    annotations, annotation_idx = append_annotations(result,result_,right_coord,annotations,image_idx,annotation_idx,seg_option,(mini_x,mini_y))
                    image_idx += 1
    return images, annotations

In [ ]:
def save_json(images, annotations, mode='train'):
    coco_dict = dict(
        info= dict(
            year=2021, 
            version="1.0", 
            description="OCR", 
            contributor="", 
            url=None, 
            date_created=None
        ), # year, version, description, contributor, url, date_created
        licenses=dict(
            id=0,
            name='CC BY 4.0',
            url=''
        ), # id, name, url
        images=list(), # id, file_name, height, width
        annotations=list(), # id, image_id, category_id, bbox, area, iscrowd
        categories = [
            dict(id=0,
                name='word')
        ] # id, name, supercategory
    )
    coco_dict['images'] = images
    coco_dict['annotations'] = annotations
    coco_dict['categories'] = [{'id':0, 'name':'word','super_category':None}]
    with open(f'/content/data/{mode}.json', 'w', encoding='utf-8') as jfile:
        json.dump(coco_dict, jfile)

In [ ]:
images, annotations = make_dataset('train', seg_option=False, just_image=False)
save_json(images, annotations,'train')
del images, annotations
images, annotations = make_dataset('valid', seg_option=False, just_image=False)
save_json(images, annotations,'valid')

## 전처리 데이터 저장

In [ ]:
%cd /content/data
!zip -r /content/drive/MyDrive/input/valid.zip ./valid
!zip -r /content/drive/MyDrive/input/train.zip ./train
!cp /content/data/valid.json /content/drive/MyDrive/input/valid.json
!cp /content/data/train.json /content/drive/MyDrive/input/train.json

# 데이터 정상인지 확인

In [ ]:
mode = 'train'
# with open(f'/content/drive/MyDrive/detection/{mode}.json','r', encoding='utf-8') as f:
#     d = json.load(f)
with open(f'/content/data/{mode}.json','r',encoding='utf-8') as f:
    d = json.load(f)
for k in range(30):
    image_sample = d['images'][k]

    ann_sample = []
    for i in d['annotations']:
        if i['image_id'] == image_sample['id']:
            ann_sample.append(i)
    print(len(ann_sample))
    f_name = image_sample['file_name']
    img = plt.imread(f'/content/data/{mode}/{f_name}')
    # plt.imshow(img)
    # plt.show()
    fig, ax = plt.subplots(figsize=(12,12))
    
    ## segmentation
    # tmp = np.zeros((image_sample['height'], image_sample['width']))
    # for j in ann_sample:
    #     for m in range(0, len(j['segmentation'][0]), 2):
    #         tmp[j['segmentation'][0][m+1], j['segmentation'][0][m]] = 1
    # ax.imshow(tmp, cmap='gray')
    # plt.show()

    ## bounding box
    ax.imshow(img)
    for j in ann_sample:
        x,y,w,h = j['bbox']
        rect = patches.Rectangle((x,y),w,h, linewidth=3, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()


In [ ]:
cnt = 0
for i in range(len(d['annotations'])):
    if len(d['annotations'][i]['segmentation'][0]) <= 4:
        print(d['annotations'][i])
        image_id = d['annotations'][i]['image_id']
        img = plt.imread(f'/content/data/{mode}/{image_id:05d}.png')
        # fig, ax = plt.subplots(figsize=(12,12))
        
        ax.imshow(img)
        for j in d['annotations']:
            x,y,w,h = j['bbox']
            
            if j['image_id'] == image_id and len(j['segmentation'][0])<=4:
                print(j['segmentation'][0])
                # rect = patches.Rectangle((x,y),w,h, linewidth=3, edgecolor='r', facecolor='none')
                # ax.add_patch(rect)
                cnt += 1
                bbox = img[y:y+h, x:x+w, :]
                plt.imshow(bbox)
                plt.show()
        # plt.show()

cnt # 348개의 글자가 거의 사라짐 - mean이 문제일수도 -> 1.0 mean으로 바꾸니 3개로 줄엇다 -> mean 값으로 설정하면 검은색이 큰 경우에 
# 4개 이하 1223개 -> 맥스값의 절반은 어떨까?

# MMDetection 학습

In [ ]:
del images, annotations

In [ ]:
%cd /content/Swin-Transformer-Object-Detection/

/content/Swin-Transformer-Object-Detection


In [ ]:
import sys
from mmcv import Config
from mmcv.runner import load_checkpoint
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, set_random_seed, init_detector
from mmdet.datasets import build_dataloader, build_dataset, replace_ImageToTensor
from mmdet.utils import collect_env, get_root_logger

import torch
import time
import json

import os

In [ ]:
classes = (
    "word",
)
CUR_PATH = os.getcwd()
CFG_PATH = "/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/detection/configs/cascade_rcnn/cascade_rcnn_swin_tiny_fpn_1x_coco.py" # cascade_rcnn_swin_tiny_fpn_1x_coco.py / cascade_rcnn_r101_fpn_20e_coco.py
# CFG_PATH = "/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/detection/configs/swin/cascade_mask_rcnn_swin_base_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco.py"
# CFG_PATH = "/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/detection/configs/mask_rcnn/mask_rcnn_r50_fpn_1x_coco.py"
PREFIX = "/content/data/image"
WORK_DIR = "/content/drive/MyDrive/Colab Notebooks/Computer_Vision/work_dir"
# CHK_PATH = "/content/drive/MyDrive/Colab Notebooks/lesion_obd/cascade_mask_rcnn_swin_small_patch4_window7.pth"
# config file 들고오기
cfg = Config.fromfile(CFG_PATH)

In [ ]:
for i in cfg:
    print(i, '*'*40)
    print(cfg[i])

In [ ]:

cfg.data.train.classes = classes
cfg.data.train.img_prefix = "/content/data/train"
cfg.data.train.ann_file = '/content/data/train.json'

cfg.data.val.classes = classes
cfg.data.val.img_prefix = "/content/data/valid"
cfg.data.val.ann_file = '/content/data/valid.json'

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 4

cfg.seed = 42
cfg.gpu_ids = [0]

distributed = False
cfg.model.backbone = {'type': 'SwinTransformer', 'embed_dim': 128, 'depths': [2, 2, 18, 2], 'num_heads': [4, 8, 16, 32], 'window_size': 12, 'mlp_ratio': 4.0, 'qkv_bias': True, 'qk_scale': None, 'drop_rate': 0.0, 'attn_drop_rate': 0.0, 'drop_path_rate': 0.2, 'ape': False, 'patch_norm': True, 'out_indices': (0, 1, 2, 3), 'use_checkpoint': False}
cfg.model.neck = {'type': 'FPN', 'in_channels': [128, 256, 512, 1024], 'out_channels': 256, 'num_outs': 5}

cfg.work_dir = WORK_DIR
cfg.runner.max_epochs = 20
cfg.rtotal_epochs = 20
cfg.optimizer = dict(type='Adam', lr=0.0001, weight_decay=0.0001)

cfg.lr_config = dict(
    policy='CosineAnnealing', # The policy of scheduler, also support CosineAnnealing, Cyclic, etc. Refer to details of supported LrUpdater from https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/lr_updater.py#L9.
    by_epoch=False,
    warmup='linear', # The warmup policy, also support `exp` and `constant`.
    warmup_iters=1000, # The number of iterations for warmup
    warmup_ratio=0.001, # The ratio of the starting learning rate used for warmup
    min_lr=1e-07)

cfg.log_config.interval = 600
cfg.checkpoint_config.interval = 1
cfg.log_config = {'hooks': [{'type': 'TextLoggerHook'}], 'interval': 600}

# cfg.resume_from = 

# cfg.load_from = '/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/detection/pth/swin_base_patch4_window12_384.pth'
# cfg.load_from = '/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/detection/pth/cascade_mask_rcnn_swin_base_patch4_window7.pth'
# cfg.load_from = '/content/drive/MyDrive/Colab Notebooks/Computer_Vision/work_dir/87.3_200_under.pth'
# cfg.optimizer_config.use_fp16 = False
# cfg.model.pretrained = None
# cfg.fp16 = None
model = build_detector(cfg.model)

datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES

loading annotations into memory...
Done (t=14.95s)
creating index...
index created!


In [ ]:
train_detector(model, datasets[0], cfg, distributed=distributed, validate=True)

# Inference

In [ ]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [ ]:
cfg.data.test.classes = classes
cfg.data.test.img_prefix = '/content/data/valid'
cfg.data.test.ann_file = '/content/data/valid.json'

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))

checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/Computer_Vision/work_dir/epoch_7.pth'
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

output = single_gpu_test(model,data_loader, show_score_thr=0.05)

predictio_strings = []
file_names = []


loading annotations into memory...
Done (t=5.99s)
creating index...
index created!
load checkpoint from local path: /content/drive/MyDrive/Colab Notebooks/Computer_Vision/work_dir/epoch_7.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 3521/3521, 6.7 task/s, elapsed: 529s, ETA:     0s

In [ ]:
img_ = plt.imread(sorted(glob('/content/data/valid/*'))[0]) # 

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
ax.imshow(img_)

for idx, i in enumerate(output[0][0]):
    # x,y,w,h,confi = i
    x,y,w,h,confi = i
    rect = patches.Rectangle((x,y),w-x,h-y, linewidth=3, edgecolor='r', facecolor='none')
    ax.add_patch(rect)



In [ ]:
# single infernece
from mmdet.apis import init_detector, inference_detector
import mmcv

# Specify the path to model config and checkpoint file
config_file = '/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/detection/configs/cascade_rcnn/cascade_rcnn_swin_tiny_fpn_1x_coco.py'
checkpoint_file = '/content/drive/MyDrive/Colab Notebooks/Computer_Vision/work_dir/epoch_7.pth'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# test a single image and show the results
img = '/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/detection/tests/0.png'  # or img = mmcv.imread(img), which will only load it once
result = inference_detector(model, img)
# visualize the results in a new window
print(result)
# or save the visualization results to image files
# model.show_result(img, result, out_file='result.jpg')


In [ ]:
img = plt.imread('/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/detection/tests/0.png')
fig, ax = plt.subplots(figsize=(15,15))
ax.imshow(img)
file_idx = 0
for i in result[0]:
    x,y,w,h,con = i
    if con > 0.1:
        rect = patches.Rectangle((x,y),w-x,h-y, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        print(x,y,w,h)
        pp = img[int(y):int(h), int(x):int(w), :]
        plt.imsave(f'/content/drive/Othercomputers/내 컴퓨터/workspace/ocr_ancient_korean/recognition/test/{file_idx:03d}.png', pp)
        file_idx += 1
        if file_idx == 2:
            break
    # break